# Tools and Agents

In [1]:
## using inbuild tools
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=250)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
wikipedia_tool.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv_tool.name

'arxiv'

In [4]:
tools = [arxiv_tool,wikipedia_tool]

In [5]:
### Let us create a custom tool[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100).split_documents(docs)
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever = vectordb.as_retriever()
retriever

/home/abc/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/abc/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x757de18b9d00>)

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"Langsmith Search","Search anything about langsmit")
retriever_tool.name

'Langsmith Search'

In [8]:
tools = [arxiv_tool,wikipedia_tool,retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/abc/anaconda3/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith Search', description='Search anything about langsmit', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x757eaf2f4fe0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x757de18b9d00>), document_prompt=Promp

In [9]:
### Run all this tools with agents and LLM models
### Tools,LLM --> AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

import os

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key,model="LLama3-8b-8192")



In [10]:
### Prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/abc/anaconda3/lib/python3.12/site-packages/langsmith/client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
## Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [12]:
## Agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent,tools = tools, verbose= True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [13]:
agent_executor.invoke({"input":"Tell me about langsmith"})




> Entering new AgentExecutor chain...

Invoking: `Langsmith Search` with `{'query': 'what is langsmith'}`


Get started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

is not necessary - LangSmith works on its own!1. Install LangSmithâ€‹PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API keyâ€‹To create an API key head to the Settings page. Then click Create API Key.3. Set up your environmentâ€‹Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first traceâ€‹Tracing to LangSmith for

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you

{'input': 'Tell me about langsmith',
 'output': "It looks like you've already got a good understanding of LangSmith! It seems like a platform for building production-grade LLM applications, allowing you to monitor and evaluate your application closely. LangChain is mentioned, but it appears to be optional, and LangSmith can work on its own. \n\nLet me summarize the key points for you:\n\n1. Installation: LangSmith can be installed using pip or yarn.\n2. API Key: You need to create an API key on the Settings page.\n3. Environment Setup: You need to set up your environment by exporting certain variables, such as LANGCHAIN_TRACING_V2 and LANGCHAIN_API_KEY.\n4. Logging Your First Trace: You can log your first trace to LangSmith to get started.\n5. Running Your First Evaluation: You can run your first evaluation using LangSmith.\n\nIt's great that you're already familiar with LangSmith! If you have any more questions or need further clarification, feel free to ask!"}

In [14]:
agent_executor.invoke({"input":"What is wikipedia"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'What is Wikipedia?'}`


Page: Wikipedia
Summary: Wikipedia is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is the largest and most-read ref
Invoking: `wikipedia` with `{'query': 'What is Wikipedia?'}`


Page: Wikipedia
Summary: Wikipedia is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is the largest and most-read refWikipedia is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is the largest and most-read reference work in the world, with over 50 million articles across many languages. It is a reliable source of i

{'input': 'What is wikipedia',
 'output': 'Wikipedia is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is the largest and most-read reference work in the world, with over 50 million articles across many languages. It is a reliable source of information on a wide range of topics, including history, science, technology, culture, and more.'}

In [15]:
agent_executor.invoke({"input":"what is the research paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cBased on the output from the tool call, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023. The paper is authored by a team of researchers led by Ashish Vaswani and discusses sequence transduction models, specifically the dominant models being based on convolutional neural networks (CNNs).

> Finished chain.


{'input': 'what is the research paper 1706.03762 about?',
 'output': 'Based on the output from the tool call, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023. The paper is authored by a team of researchers led by Ashish Vaswani and discusses sequence transduction models, specifically the dominant models being based on convolutional neural networks (CNNs).'}